<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=f6082464b60b152824f3d6a2e1b2abd66b7b4bba2cc93b54c9fc9bfcdebe37ce
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-28 12:26:00
-------------------
qualified stocks: 91
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  9.00 L
Current:  1.37 C
-------------------
Today PnL: -35.50 K (-0.26%)
Current PnL: -20.23 L (-13.6%)
CY Booked + Current PnL: -11.19 L (-7.52%)
-------------------
Total profit:  2.95 L
Total loss:  -23.18 L
-------------------
Total Booked + Current PnL: 15.79 L (12.98%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.6%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.16 C
Est FTT PnL: 78.62 L (57.37%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.55%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-18.25,57.0,H-LC,7.50,112832.0,8520.0,8135.0,-0.07,8.17,7.21,15.96,9.0,1.05,0.88,26.45,X5K,ATH,METALS
53,MEDANTA,1486.00,14.90,64.0,H-MC,14.85,164082.0,37882.0,8286.0,1.04,30.02,5.05,36.59,108.0,4.57,1.28,46.35,XY24,NTT,HEALTHCARE
4,APOLLOHOSP,8285.00,-14.19,62.0,H-LC,8.30,171545.0,15368.0,10722.0,0.05,9.84,6.25,16.71,22.0,1.43,1.34,28.99,X40N,BTT,HEALTHCARE
66,SBIN,863.00,-13.25,44.0,M-LC,4.08,214120.0,12640.0,14582.0,0.02,6.27,6.81,13.51,60.0,0.87,1.67,19.66,XY25,NTT,BANKS
81,TTKPRESTIG,770.00,97.01,55.0,M-SC,8.42,84842.0,-15935.0,16027.0,1.17,-15.81,18.89,0.09,245.0,-0.99,0.66,11.46,OX40N,NTT,DURABLES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,5211.76,-9.59,35.0,H-SC,6.15,102785.0,781.0,43139.0,-1.48,0.77,41.97,43.06,122.0,0.02,0.8,18.01,X40N,ATH,MISC
59,RAJOOENG,143.33,-32.60,38.0,H-SC,9.55,102200.0,-300.0,41125.0,-1.19,-0.29,40.24,39.83,114.0,-0.01,0.8,1.19,AR,ATH,MISC


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-42.59,32.0,H-LC,13.17,195078.0,-21441.0,114355.0,-0.40,-9.90,58.62,42.91,7.0,-0.19,1.52,2.16,AR,NTT,ELECTRICAL
84,UNITDSPR,1644.00,-15.51,37.0,H-LC,8.75,169844.0,-11915.0,47166.0,-1.24,-6.56,27.77,19.39,86.0,-0.25,1.33,1.06,X40N,NTT,BREWERIES
45,ITC,452.00,-39.19,38.0,H-LC,3.13,146164.0,-3768.0,18358.0,-0.45,-2.51,12.56,9.74,4.0,-0.21,1.14,3.57,X40,NTT,FMCG
33,HCLTECH,1943.91,-0.22,38.0,H-LC,10.38,177132.0,-13514.0,60030.0,-2.74,-7.09,33.89,24.40,8.0,-0.23,1.38,7.72,X40,ATH,IT
47,JIOFIN,387.00,-0.42,42.0,H-LC,15.45,220290.0,2884.0,50601.0,-0.33,1.33,22.97,24.61,48.0,0.06,1.72,56.87,XY24,BTT,FINANCE


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LTIM,7230.2,-7.23,46.0,H-LC,3.11,183816.0,-16494.0,76467.0,-1.15,-8.23,41.60,29.94,16.0,-0.22,1.44,26.99,X200,ATH,IT
45,ITC,452.0,-39.19,38.0,H-LC,3.13,146164.0,-3768.0,18358.0,-0.45,-2.51,12.56,9.74,4.0,-0.21,1.14,3.57,X40,NTT,FMCG
55,NESTLEIND,1377.0,-11.53,53.0,H-LC,3.17,272399.0,6973.0,49822.0,0.16,2.63,18.29,21.40,11.0,0.14,2.13,10.62,XY25,NTT,FMCG
20,CIPLA,1795.0,-19.09,60.0,H-LC,4.23,216145.0,11645.0,29763.0,-0.13,5.69,13.77,20.25,10.0,0.39,1.69,14.92,X40N,BTT,PHARMA
5,ASIANPAINT,4250.0,-12.28,57.0,H-LC,6.75,171047.0,-30339.0,117954.0,1.37,-15.07,68.96,43.51,27.0,-0.26,1.34,19.07,X40,BTT,PAINTS


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,HONAUT,58357.33,-17.33,57.0,H-SC,3.19,116565.0,-11373.0,58504.0,2.55,-8.89,50.19,36.84,143.0,-0.19,0.91,19.33,X40N,ATH,ELECTRICAL
75,TANLA,1963.11,-27.72,51.0,H-SC,8.08,165474.0,-73941.0,346900.0,2.12,-30.88,209.64,114.01,133.0,-0.21,1.29,51.77,AR,ATH,IT
14,BLUESTARCO,2326.38,10.24,60.0,H-MC,4.56,190300.0,25630.0,42342.0,2.04,15.56,22.25,41.28,89.0,0.61,1.49,24.79,X40N,ATH,AC
74,SYMPHONY,1306.00,-41.52,33.0,M-SC,17.00,124450.0,-46691.0,46631.0,2.02,-27.28,37.47,-0.03,196.0,-1.00,0.97,2.02,OX40N,NTT,DURABLES
13,BERGEPAINT,680.00,-19.46,43.0,H-MC,8.51,216310.0,-11029.0,59096.0,1.71,-4.85,27.32,21.14,106.0,-0.19,1.69,21.92,XY24,NTT,PAINTS


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HCLTECH,1943.91,-0.22,38.0,H-LC,10.38,177132.0,-13514.0,60030.0,-2.74,-7.09,33.89,24.40,8.0,-0.23,1.38,7.72,X40,ATH,IT
40,INDIAMART,4850.92,-50.91,50.0,H-SC,12.43,137153.0,13817.0,119940.0,-2.52,11.20,87.45,108.45,119.0,0.12,1.07,36.06,AR,ATH,MISC
61,REPCOHOME,880.00,-62.00,24.0,H-SC,7.21,139419.0,-65579.0,210815.0,-2.10,-31.99,151.21,70.85,134.0,-0.31,1.09,14.54,XY24,NTT,FINANCE
67,SFL,1287.00,38.02,58.0,M-SC,10.00,182238.0,-79980.0,149800.0,-1.99,-30.50,82.20,26.63,229.0,-0.53,1.42,13.31,XY24,NTT,MISC
79,TITAGARH,1548.00,-8.89,49.0,H-SC,9.52,149275.0,-42740.0,121629.0,-1.88,-22.26,81.48,41.08,158.0,-0.35,1.17,26.34,XY24,NTT,ENGINEERING


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TTKPRESTIG,770.0,97.01,55.0,M-SC,8.42,84842.0,-15935.0,16027.0,1.17,-15.81,18.89,0.09,245.0,-0.99,0.66,11.46,OX40N,NTT,DURABLES
82,UJJIVANSFB,60.0,82.03,40.0,H-SC,15.18,114723.0,-27756.0,47277.0,-0.70,-19.48,41.21,13.70,163.0,-0.59,0.90,35.40,OX40N,NTT,BANKS
48,KANSAINER,340.0,-69.07,46.0,H-SC,3.38,212976.0,-56691.0,93028.0,1.10,-21.02,43.68,13.47,138.0,-0.61,1.66,7.72,XY24,NTT,PAINTS
41,INDIGOPNTS,1408.0,110.38,42.0,M-SC,1.96,137814.0,-36745.0,36783.0,0.69,-21.05,26.69,0.02,221.0,-1.00,1.08,19.87,OX40N,NTT,PAINTS
19,CERA,9475.0,-21.24,38.0,H-SC,3.01,112500.0,-31303.0,58050.0,-0.99,-21.77,51.60,18.60,149.0,-0.54,0.88,24.15,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TCS,4998.00,-26.76,50.0,H-LC,15.74,242611.0,-53323.0,147241.0,-1.48,-18.02,60.69,31.73,1.0,-0.36,1.90,3.58,X40,BTT,IT
43,INFY,2275.00,-17.80,51.0,H-LC,11.82,269001.0,7400.0,138213.0,-1.75,2.83,51.38,55.67,3.0,0.05,2.10,9.11,X40,BTT,IT
45,ITC,452.00,-39.19,38.0,H-LC,3.13,146164.0,-3768.0,18358.0,-0.45,-2.51,12.56,9.74,4.0,-0.21,1.14,3.57,X40,NTT,FMCG
87,VBL,671.64,-10.56,52.0,H-LC,8.52,271448.0,6408.0,89904.0,-0.79,2.42,33.12,36.33,5.0,0.07,2.12,16.04,X40N,ATH,FMCG
1,ABB,7934.00,-42.59,32.0,H-LC,13.17,195078.0,-21441.0,114355.0,-0.40,-9.90,58.62,42.91,7.0,-0.19,1.52,2.16,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.0,14.90,64.0,H-MC,14.85,164082.0,37882.0,8286.0,1.04,30.02,5.05,36.59,108.0,4.57,1.28,46.35,XY24,NTT,HEALTHCARE
27,DMART,5201.0,-7.76,72.0,H-LC,11.93,230413.0,45932.0,24447.0,-0.30,24.90,10.61,38.14,38.0,1.88,1.80,38.14,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,14.90,64.0,H-MC,14.85,164082.0,37882.0,8286.0,1.04,30.02,5.05,36.59,108.0,4.57,1.28,46.35,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,10.24,60.0,H-MC,4.56,190300.0,25630.0,42342.0,2.04,15.56,22.25,41.28,89.0,0.61,1.49,24.79,X40N,ATH,AC
34,HEROMOTOCO,5949.07,-2.52,72.0,H-MC,14.98,180432.0,29519.0,27787.0,1.61,19.56,15.40,37.97,93.0,1.06,1.41,49.81,AR,ATH,AUTO
88,VOLTAS,1918.49,-2.62,57.0,H-MC,6.53,204180.0,12438.0,83591.0,1.03,6.49,40.94,50.08,99.0,0.15,1.59,13.73,XY25,ATH,AC
24,DABUR,735.00,-6.72,54.0,H-MC,3.86,203014.0,5986.0,83642.0,-0.35,3.04,41.20,45.49,102.0,0.07,1.59,16.47,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,-2.52,72.0,H-MC,14.98,180432.0,29519.0,27787.0,1.61,19.56,15.40,37.97,93.0,1.06,1.41,49.81,AR,ATH,AUTO
40,INDIAMART,4850.92,-50.91,50.0,H-SC,12.43,137153.0,13817.0,119940.0,-2.52,11.20,87.45,108.45,119.0,0.12,1.07,36.06,AR,ATH,MISC
89,WHIRLPOOL,2270.00,-42.56,46.0,M-SC,5.63,97162.0,5664.0,73085.0,0.82,6.19,75.22,86.07,223.0,0.08,0.76,40.56,XR,NTT,DURABLES
90,WIPRO,420.00,-11.07,54.0,M-LC,7.76,156259.0,5314.0,104147.0,-0.71,3.52,66.65,72.51,53.0,0.05,1.22,9.75,XR,NTT,IT
59,RAJOOENG,143.33,-32.60,38.0,H-SC,9.55,102200.0,-300.0,41125.0,-1.19,-0.29,40.24,39.83,114.0,-0.01,0.80,1.19,AR,ATH,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-43.66,65.0,L-SC,28.14,82733.0,-30816.0,70869.0,0.30,-27.14,85.66,35.27,268.0,-0.43,0.65,102.49,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-5.35,38.0,H-SC,16.26,88042.0,-12921.0,107446.0,-0.98,-12.80,122.04,93.63,148.0,-0.12,0.69,31.14,SR,ATH,CHEMICALS
52,MASFIN,398.61,-18.70,49.0,H-SC,12.63,92625.0,-5355.0,26954.0,0.46,-5.47,29.10,22.05,152.0,-0.20,0.72,34.64,XR,ATH,FINANCE
35,HINDALCO,756.01,-18.25,57.0,H-LC,7.50,112832.0,8520.0,8135.0,-0.07,8.17,7.21,15.96,9.0,1.05,0.88,26.45,X5K,ATH,METALS
82,UJJIVANSFB,60.00,82.03,40.0,H-SC,15.18,114723.0,-27756.0,47277.0,-0.70,-19.48,41.21,13.70,163.0,-0.59,0.90,35.40,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-43.66,65.0,L-SC,28.14,82733.0,-30816.0,70869.0,0.30,-27.14,85.66,35.27,268.0,-0.43,0.65,102.49,XR,NTT,HOTELS
34,HEROMOTOCO,5949.07,-2.52,72.0,H-MC,14.98,180432.0,29519.0,27787.0,1.61,19.56,15.40,37.97,93.0,1.06,1.41,49.81,AR,ATH,AUTO
27,DMART,5201.00,-7.76,72.0,H-LC,11.93,230413.0,45932.0,24447.0,-0.30,24.90,10.61,38.14,38.0,1.88,1.80,38.14,X40N,NTT,FMCG
14,BLUESTARCO,2326.38,10.24,60.0,H-MC,4.56,190300.0,25630.0,42342.0,2.04,15.56,22.25,41.28,89.0,0.61,1.49,24.79,X40N,ATH,AC
6,ASIANTILES,137.00,6733.33,49.0,L-SC,6.38,75001.0,-18809.0,95431.0,-0.30,-20.05,127.24,81.67,269.0,-0.20,0.59,45.24,XR,NTT,CERAMICS


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.92
1,25,42.05
2,50,72.26


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.32
LC    34.51
MC    27.21
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.23
X40N     14.42
X40      14.13
XY25     10.03
XR        9.25
AR        8.74
OX40N     7.62
X5K       2.32
MH        1.73
X200      1.44
SR        1.13
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.05
H-MC    24.24
H-SC    23.11
M-SC    13.60
M-LC     7.38
M-MC     2.64
L-SC     1.61
L-LC     1.08
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.93,-1.65,35.30
IT,12.39,-17.67,76.44
BANKS,7.63,-16.63,66.15
FINANCE,7.49,-25.79,72.17
MISC,6.05,-18.59,86.56
PAINTS,5.77,-18.26,41.57
ELECTRICAL,4.89,-12.18,60.76
AUTO,4.72,-11.88,60.55
AC,3.84,1.54,31.80


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2897010.0,23
XR,1109253.0,13
AR,1089693.0,10
X40,793368.0,10
X40N,521040.0,11
OX40N,484168.0,10
XY25,457629.0,7
SR,198992.0,2
X5K,140848.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2817209.0,27
M-SC,1505696.0,17
H-MC,1275695.0,18
H-LC,1127943.0,17
M-LC,433004.0,5
M-MC,339626.0,2
L-SC,256772.0,3
L-MC,59831.0,1
L-LC,45945.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      952223.0      6
M-SC       XY24      881752.0      7
H-SC       AR        664620.0      5
           XR        534718.0      6
H-MC       XY24      491246.0      5
H-LC       X40       481796.0      5
M-MC       XY24      339626.0      2
H-LC       AR        236981.0      2
H-MC       X40       226856.0      4
H-SC       OX40N     211208.0      4
H-LC       X40N      202002.0      5
H-MC       XY25      199743.0      2
H-SC       SR        198992.0      2
           X40N      194548.0      3
M-LC       XY24      181562.0      2
L-SC       XR        166300.0      2
M-SC       AR        160305.0      2
M-LC       X5K       132713.0      1
M-SC       OX40N     129905.0      4
           XY25      125398.0      1
H-MC       X40N      124490.0      3
M-SC       XR        123620.0      2
H-MC       XR        120637.0      1
M-LC       XR        104147.0      1
L-SC       OX40N      90472.0      1
M-SC       X40        84716.0      1
H-LC       X200       76467.0      1
           XY25       71961.0      2
H-SC       MH         60900.0      1
L-MC       XR         59831.0      1
H-MC       OX40N      52583.0      1
H-LC       XY24       50601.0      1
L-LC       XY25       45945.0      1
H-MC       MH         32353.0      1
           AR         27787.0      1
M-LC       XY25       14582.0      1
H-LC       X5K         8135.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
